In [18]:
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist # 앞으로는 이렇게 mnist를 호출하면 됌.


(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
Y_one_hot = np.eye(10)[Y_train]
Y_test_one_hot = np.eye(10)[Y_test]

sess = tf.Session()

%matplotlib inline  
print ("packs loaded")

packs loaded


In [3]:
Y_one_hot.shape
Y_test_one_hot.shape

(10000, 10)

# CREATE TENSOR GRAPH FOR LOGISTIC REGRESSION


In [74]:
x = tf.placeholder("float", [None, 784])
y = tf.placeholder("float", [None, 10])

W = tf.Variable(tf.random_normal([784,10], stddev= 0.1)) 
b = tf.Variable(tf.zeros([10]))

# 출력해보기
init = tf.global_variables_initializer()
sess.run(init)
sess.run(W)
sess.run(b)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [95]:
# Logistic regression model
actv = tf.nn.softmax(tf.matmul(x,W) + b)

# cost
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(actv)))
cost = -tf.reduce_sum(y*tf.log(tf.clip_by_value(actv,1e-10,1.0)))
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(tf.clip_by_value(actv,1e-10,1.0))))


# optimizer
learning_rate = 0.01
optm = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)


# PREDICTION AND ACCURACY

In [96]:
pred = tf.equal(tf.argmax(actv,1), tf.argmax(y,1))
accr = tf.reduce_mean(tf.cast(pred, "float"))

In [97]:
init = tf.initialize_all_variables()

# TRAIN MODEL


In [98]:
training_epochs = 20
batch_size = 100
display_step = 5

In [100]:
# session
sess = tf.Session()
sess.run(init)

In [101]:


# MINI-BATCH Learning
for epoch in range(training_epochs):
    avg_cost = 0.
    num_batch = len(X_train)//batch_size
    
    for i in range(num_batch):
        batch_xs = X_train[batch_size*i:batch_size*(i+1)] 
        batch_xs = batch_xs.reshape(batch_size,-1) # size 바꿔주기
        batch_ys = Y_one_hot[batch_size*i:batch_size*(i+1)] # 

        # 학습
        sess.run(optm, feed_dict={x: batch_xs, y: batch_ys})
        feeds = {x: batch_xs, y: batch_ys}
        avg_cost += sess.run(cost, feed_dict=feeds)/num_batch
        
    if epoch % display_step == 0:
        feeds_train = {x: batch_xs, y: batch_ys}
        
        X_test = X_test.reshape(-1,28*28)
        feeds_test = {x: X_test, y: Y_test_one_hot}
        
        train_acc = sess.run(accr, feed_dict = feeds_train)
        test_acc = sess.run(accr, feed_dict = feeds_test)
        
        print("Epoch: %03d/%03d cost: %.9f train_acc: %.3f test_acc: %.3f" 
               % (epoch, training_epochs, avg_cost, train_acc, test_acc))
print ("DONE")        
        

Epoch: 000/020 cost: 2078.390075480 train_acc: 0.100 test_acc: 0.098
Epoch: 005/020 cost: 2078.390075480 train_acc: 0.100 test_acc: 0.098
Epoch: 010/020 cost: 2078.390075480 train_acc: 0.100 test_acc: 0.098
Epoch: 015/020 cost: 2078.390075480 train_acc: 0.100 test_acc: 0.098
DONE


In [93]:
train_acc

0.1

In [72]:
sess.run(actv, feed_dict= {x: batch_xs})

test = tf.matmul(x, W) + b

sess.run(test, {x:batch_xs}).shape


(100, 10)

In [49]:
num_batch = len(X_train)//batch_size
for i in range(num_batch):
    batch_xs = X_train[batch_size*i:batch_size*(i+1)]
    batch_ys = Y_one_hot[batch_size*i:batch_size*(i+1)]
    
    # 학습
    sess.run(optm, feed_dict={x: batch_xs, y: batch_ys})
    feeds = {x: batch_xs, y: batch_ys}
    avg_cost += sess.run(cost, feed_dict=feeds)/num_batch



ValueError: Cannot feed value of shape (100, 28, 28) for Tensor 'Placeholder_3:0', which has shape '(?, 784)'